# Hey there! I made just one notebook for the whole assignment, so please scrolldown to find all questions. Thank you.

### Q1: Use pandas, or the BeautifulSoup package, or any other way you are comfortable with to transform the data in the table on the Wikipedia page into the above pandas dataframe.

#### At first we import the libraries required

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

#### Scrapping Postal Code's List from Wikipage using Beautiful Soup 

In [2]:
# download url data from internet
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')

#### Converting to dataframe

In [3]:
#Creating dataframe
df = pd.DataFrame(columns = ['Postalcode','Borough','Neighborhood'])

# loop through wiki to find postcode, borough, neighborhood 
My_table = soup.find('div', class_='mw-parser-output').table.tbody

for tr in My_table.find_all('tr'):
    for td in tr.find_all('td'):
        postcode = td.p.b.text
        borough = td.p.span.text
        df = df.append({'Postalcode': postcode,'Borough': borough},ignore_index=True)

# As neighborhood is into parenthesis in borough, we will extract them. 
# Additionally, if a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
df.Neighborhood = df.Borough.apply(lambda st: st[st.find("(")+1:st.find(")")])

# Now we will delete the neighborhood from the borough column
df.Borough = df.Borough.str.replace(r"\(.*\)","")

# Replace the / for ,
df.Borough = df.Borough.str.replace(r" /",",")
df.Neighborhood = df.Neighborhood.str.replace(r" /",",")

#### Cleaning the dataframe

In [4]:
# We delete the rows wehere borough is Not assigned.
df = df[df.Borough != 'Not assigned']

#reset the index
df.reset_index(drop = True, inplace = True)

#### At least we will use the .shape method to print the number of rows of the dataframe

In [5]:
df.shape

(103, 3)

In [6]:
df.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,"Queen's Park, Ontario Provincial Government","Queen's Park, Ontario Provincial Governmen"


### Q2: Get the latitude and the longitude coordinates of each neighborhood

#### Import the librarie required

In [7]:
!pip install geocoder
import geocoder

#### We define the function to get the latitude and longitude of postal codes

In [ ]:
def coordinates(postal_code):
    
    # initialize your variable to None
    lat_lng_coords = None
    
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]

# we do a test with postal code M3A    
coordinates('M3A')

#### the package was not able to get the geographical coordinates of the neighborhoods using the Geocoder package
#### so we will use the link to the csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data


In [8]:
url_coord = 'https://cocl.us/Geospatial_data'
df_coord = pd.read_csv(url_coord)
df_coord.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Now we will merge both dataframes

In [17]:
# at first we have to rename one dataframe to match in the postal code name column
df = df.rename(columns={"Postalcode": "Postal Code"})

# once the dataframes have the same name, we can merge them
df_merged = pd.merge(df, df_coord, on = 'Postal Code')
print(df_merged.shape)
df_merged.head(5)

(103, 5)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,"Queen's Park, Ontario Provincial Government","Queen's Park, Ontario Provincial Governmen",43.662301,-79.389494


### Q3: Explore and cluster the neighborhoods in Toronto

#### We download all dependencies required

In [18]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1e             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

#### We decide to work with only boroughs that contain the word Toronto

In [23]:
df_toronto = df_merged[df_merged['Borough'].str.contains("Toronto")]
print(df_toronto.shape)
df_toronto.head(5)

(39, 5)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


#### we define our Foursquare credentials and version

In [24]:
CLIENT_ID = 'TAG5RZ1QHY4ZXDMSIWTTRVY4HXE5NC1YADW1VGM1RGNRAWCZ' # your Foursquare ID
CLIENT_SECRET = 'N0BTELUTQQ3SHDALD3ZKMYBF51QSM4ZGAUFAAULTJFHJOF0O' # your Foursquare Secret
VERSION = '20180604'

#### We will define the function to get nearby venues

In [25]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### We run the get nearby venues function with previous dataframe

In [41]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )
toronto_venues.head(5)

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


#### Now we will check the most common venues in each neighborhood

In [34]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.drop(['Neighborhood'],axis=1,inplace=True) 
toronto_onehot.insert(loc=0, column='Neighborhood', value=toronto_venues['Neighborhood'] )

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()


def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Beer Bar,Farmers Market,Cocktail Bar,Seafood Restaurant,Bakery,Cheese Shop,Café,Restaurant,Irish Pub
1,"Brockton, Parkdale Village, Exhibition Place",Nightclub,Breakfast Spot,Café,Coffee Shop,Pet Store,Stadium,Burrito Place,Restaurant,Climbing Gym,Yoga Studio
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Boutique,Bar,Sculpture Garden,Plane,Boat or Ferry,Harbor / Marina
3,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Middle Eastern Restaurant,Japanese Restaurant,Burger Joint,Ice Cream Shop,Juice Bar,Department Store,Thai Restaurant
4,Christie,Grocery Store,Café,Park,Nightclub,Italian Restaurant,Diner,Restaurant,Baby Store,Candy Store,Gas Station


#### Clusters for neighborhoods

In [35]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [39]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Berczy Park,Coffee Shop,Beer Bar,Farmers Market,Cocktail Bar,Seafood Restaurant,Bakery,Cheese Shop,Café,Restaurant,Irish Pub
1,1,"Brockton, Parkdale Village, Exhibition Place",Nightclub,Breakfast Spot,Café,Coffee Shop,Pet Store,Stadium,Burrito Place,Restaurant,Climbing Gym,Yoga Studio
2,1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Boutique,Bar,Sculpture Garden,Plane,Boat or Ferry,Harbor / Marina
3,1,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Middle Eastern Restaurant,Japanese Restaurant,Burger Joint,Ice Cream Shop,Juice Bar,Department Store,Thai Restaurant
4,1,Christie,Grocery Store,Café,Park,Nightclub,Italian Restaurant,Diner,Restaurant,Baby Store,Candy Store,Gas Station


#### Creating the map

In [40]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters